In [1]:
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import numpy as np
import scipy.io as sio

In [2]:
 # Loads Moby Dick
mobydick = strip_headers(load_etext(2701)).strip()
text = [mobydick.split('ETYMOLOGY')[-1]]

In [3]:
# Loads Pride and Prejudice
pride = strip_headers(load_etext(1342)).strip()
text.append(pride.split('Austen')[-1])

In [4]:
# Loads Huck Finn
huckfinn = strip_headers(load_etext(76)).strip()
text.append(huckfinn.split('\nEXPLANATORY\n\n')[-1])

In [5]:
# Loads tom sawyer
toms = strip_headers(load_etext(74)).strip()
text.append(toms.split('\nPREFACE\n\n')[-1])

In [6]:
# Loads Sense and Sensibility
sense = strip_headers(load_etext(161)).strip()
text.append(sense.split('\n(1811)\n\n')[-1])

In [7]:
# Loads Twenty Thousand Leagues
leagues20k = strip_headers(load_etext(164)).strip()
text.append(leagues20k.split(' VERNE\n\n')[-1])

In [8]:
# Loads Ulysses
ulysses = strip_headers(load_etext(4300)).strip()
text.append(ulysses.split('\n— I —\n')[-1])

In [18]:
titles = ['Moby Dick','Pride and Prejudice','Huck Finn','Tom Sawyer','Sense and Sensibility','20K Leagues','Ulysses']

In [10]:
print(ulysses.split('\n— I —\n')[-1][:1000])



[ 1 ]

Stately, plump Buck Mulligan came from the stairhead, bearing a bowl of
lather on which a mirror and a razor lay crossed. A yellow
dressinggown, ungirdled, was sustained gently behind him on the mild
morning air. He held the bowl aloft and intoned:

—_Introibo ad altare Dei_.

Halted, he peered down the dark winding stairs and called out coarsely:

—Come up, Kinch! Come up, you fearful jesuit!

Solemnly he came forward and mounted the round gunrest. He faced about
and blessed gravely thrice the tower, the surrounding land and the
awaking mountains. Then, catching sight of Stephen Dedalus, he bent
towards him and made rapid crosses in the air, gurgling in his throat
and shaking his head. Stephen Dedalus, displeased and sleepy, leaned
his arms on the top of the staircase and looked coldly at the shaking
gurgling face that blessed him, equine in its length, and at the light
untonsured hair, grained and hued like pale oak.

Buck Mulligan peeped an instant under the mirror and then

In [11]:
chapters = []
for i in range(len(text)):
    chapters.append(text[i].split('CHAPTER '))
chapters[1] = text[1].split('Chapter ') # fix
chapters[6] = text[6].split(' ]\n\n') # fix
allchaps = [y for x in chapters for y in x]

In [12]:
print(chapters[6][-1][:200])

Yes because he never did a thing like that before as ask to get his
breakfast in bed with a couple of eggs since the _City Arms_ hotel when
he used to be pretending to be laid up with a sick voice doi


In [13]:
chapix = [len(x) for x in chapters]

In [14]:
def textcounter(chapters,n_features=1000):
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                       max_features=n_features,
                                       stop_words='english')
    freq = tfidf_vectorizer.fit_transform(chapters)
    words = tfidf_vectorizer.get_feature_names()
    return freq, words

In [20]:
freq, words = textcounter(allchaps,50)

In [21]:
words[freq.sum(0).argmax()]

'whale'

In [22]:
sio.savemat('/Users/zaharia/books.mat',{'freq':freq.todense(),'words':np.array(words,dtype=np.object),\
                                        'chapix':chapix,'titles':np.array(titles,dtype=np.object)})

In [31]:
sepfreq = []
sepwords = []
for i in range(len(titles)):
    tmpfreq, tmpwords = textcounter(chapters[i],50)
    sepfreq.append(tmpfreq)
    sepwords.append(tmpwords)

IndexError: list assignment index out of range